In [1]:
### all the necessary imports

# python script imports
import sys
sys.path.insert(0, '../scripts')
from download import *
from sparkshape import *

# This is copied from tutorial 1
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

# other imports
from pyspark.sql.functions import col
from pyspark.sql.functions import unix_timestamp

22/08/04 22:01:07 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/08/04 22:01:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/04 22:01:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# download the green taxi data
URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_"
YEAR = '2019'
MONTHS = range(1, 13)
TAXI_DATA = "raw/green"
download(YEAR, MONTHS, URL_TEMPLATE, TAXI_DATA)

Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03
Begin month 04
Completed month 04
Begin month 05
Completed month 05
Begin month 06
Completed month 06
Begin month 07
Completed month 07
Begin month 08
Completed month 08
Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


In [3]:
# let's import all the green taxi data for the past 6 months and take a closer look into what the data looks like
sdf_g = spark.read.parquet("../data/raw/green/")
sdf_g.printSchema()
sdf_g.show()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2018-12-22 02:17:29|  2018-12-22 02:18:57|                 N|       1.0|         264|         264|            5.0|          0.0|        3.0|  0.5|    0.

In [4]:
# Alright! Now that I've had a good idea what this data looks like, let's remove ehail_fee out of the data 
# because the values in ehail_fee are always null. I also need data for the length of each trip, and that is acquired by calculating
# the difference between lpep_pickup_datetime and lpep_dropoff_datetime.
time_diff = unix_timestamp("lpep_dropoff_datetime") - unix_timestamp("lpep_pickup_datetime")
sdf_g = sdf_g.drop(col("ehail_fee"))
sdf_g = sdf_g.withColumn("trip_duration", time_diff)

In [5]:
# save this as a new parquet file
sdf_g.write.mode("overwrite").parquet("../data/curated/green")
# to copy what tute1 said: "Your directory might look a bit funky. Don't worry, just leave it as is,
# (we don't have time to cover everything about Spark unfortunately) and you can just read in the directory as is."
# I TAKE YOUR WORD FOR THIS, OKAY!!

22/08/04 20:12:27 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/08/04 20:12:27 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/08/04 20:12:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


Now, do the same thing (yes, same exact thing) for the yellow taxi data.

In [2]:
# download the yellow taxi data
URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"
YEAR = '2019'
MONTHS = range(1, 13)
TAXI_DATA = "raw/yellow"
download(YEAR, MONTHS, URL_TEMPLATE, TAXI_DATA)

Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03
Begin month 04
Completed month 04
Begin month 05
Completed month 05
Begin month 06
Completed month 06
Begin month 07
Completed month 07
Begin month 08
Completed month 08
Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


In [6]:
sdf_y = spark.read.parquet("../data/raw/yellow/2019-01.parquet")
sdf_y.printSchema()
sdf_y.show()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+-----------

It seems like there is no ehail_fee unlike the green data. All is good to proceed to the next step.
Okay, about the next step...I collected the data from all 12 months and honestly, the total number of rows is 8.46 x 10^7 which is MASSIVE. I tried saving the entire dataset into a parquet file, and my memory straight up refused.

What do now? Well, because of this, one solution that I thought of so far is that I'll save the yellow data in a different way -- instead of merging all of them together, I'll save it month by month.

In [8]:
for i in range(1, 13):
    if i < 10:
        sdf_y = spark.read.parquet("../data/raw/yellow/2019-0" + str(i) + ".parquet")
        time_diff = unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
        sdf_y = sdf_y.withColumn("trip_duration", time_diff)
        sdf_y.write.mode("overwrite").parquet("../data/curated/yellow/2019-0" + str(i))
    else:
        sdf_y = spark.read.parquet("../data/raw/yellow/2019-" + str(i) + ".parquet")
        time_diff = unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
        sdf_y = sdf_y.withColumn("trip_duration", time_diff)
        sdf_y.write.mode("overwrite").parquet("../data/curated/yellow/2019-" + str(i))

yay! all done